<a href="https://colab.research.google.com/github/rm01243/Data-Driven-Modelling-for-AD-Workshop/blob/main/AD_ANN_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Data-Driven Modelling of Anaerobic Digesters Using Artificial Neural Networks (ANNs)

![Data Driven Modelling Image](https://raw.githubusercontent.com/rm01243/Data-Driven-Modelling-for-AD-Workshop/main/Data%20Driven%20Modelling%20Image.png)

## Workshop Overview
This notebook provides a beginner-friendly, hands-on workshop on using Artificial Neural Networks (ANNs) for modelling anaerobic digesters (AD) to predict biogas production. It is targeted at biogas researchers, AD plant operators, consultants, and sustainability professionals with little to no programming or modelling experience.

### What is Anaerobic Digestion?
Anaerobic digestion is a biological process where microorganisms break down organic material in the absence of oxygen, producing biogas (mainly methane) as a renewable energy source. Modelling this process helps predict biogas output based on inputs like temperature and substrate levels, aiding in optimisation without complex physics-based equations.

### What are ANNs?
Artificial Neural Networks are computational models inspired by the human brain. They learn patterns from data to make predictions. In this workshop, we'll use a simple ANN to predict biogas flow (`q_gas`) from AD parameters.

### Key Features:
- **No Installation Required**: Runs entirely in Google Colab, a free online tool that executes Python code in your browser.
- **Interactive**: Train a model, evaluate it, and see results with plots.
- **Data**: Uses synthetic AD data (`AD_Synthetic_Data.xlsx`) loaded directly from the GitHub repository. Synthetic data is artificially generated to mimic real AD processes for demonstration.
- **Topics Covered**: Data loading, feature selection, preprocessing, ANN building/training, evaluation, and visualisation.

### Prerequisites:
- Basic understanding of anaerobic digestion (AD) processes (e.g., what substrates and temperature affect biogas).
- Access to Google Colab (free, browser-based—no software needed).
- No coding background required—we'll explain every line!

### How to Use This Notebook:
1. Open in Colab (click the badge above).
2. Run cells sequentially (Shift+Enter or click the play button). Markdown cells (like this) explain concepts; code cells execute Python code.
3. Experiment by modifying parameters (e.g., number of epochs in training, or top features in selection) and re-run cells to see changes.
4. If something goes wrong, restart the runtime (Runtime > Restart runtime) and re-run from the top.

## Step 1: Import Libraries

**Description for Beginners**: Libraries are pre-written code packages that provide tools for tasks like data handling or modelling. We 'import' them to use their functions. For example, Pandas helps read Excel files, and Keras builds the ANN.

**Libraries Explained**:
- `numpy` (np): For numerical operations, like arrays.
- `pandas` (pd): For loading and manipulating data tables.
- `matplotlib.pyplot` (plt): For creating plots and visualisations.
- `time`: To measure how long code takes to run.
- `sklearn` modules: For splitting data, scaling, metrics, and feature selection.
- `tensorflow.keras`: For building and training the ANN (TensorFlow is the backend, Keras is the user-friendly interface).

**Why Do This?** These tools save time—no need to write everything from scratch.

**Note**: Colab has these pre-installed, so just run the cell.

In [ ]:
import numpy as np  # For numerical computations and arrays
import pandas as pd  # For reading and handling data in table format
import matplotlib.pyplot as plt  # For creating plots and graphs
import time  # To time how long the code takes
import tensorflow as tf  # For setting random seed
from sklearn.model_selection import train_test_split  # To split data into training and testing sets
from sklearn.preprocessing import MinMaxScaler  # To scale data to a 0-1 range
from sklearn.metrics import r2_score, mean_squared_error  # To evaluate model performance
from sklearn.feature_selection import mutual_info_regression  # For selecting important features
from tensorflow.keras.models import Sequential  # To build the ANN layer by layer
from tensorflow.keras.layers import Dense  # To add layers to the ANN

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

## Step 2: Load the Dataset

**Description for Beginners**: We read the Excel file from the GitHub repository into a 'DataFrame' (df), which is like an Excel table in Python. This lets us access columns easily. We'll display the first 5 rows to check the data.

**Why Do This?** To ensure the data is loaded correctly and understand its structure.

**Tips**: If the sheet name is different, change 'Sheet1'. For your own data, you can replace the URL with your file's raw GitHub URL. Update column names in the data file to full forms like 'Carbohydrates' for clarity, and adjust the 'features' list in Step 3 accordingly.

In [ ]:
data_url = 'https://raw.githubusercontent.com/rm01243/Data-Driven-Modelling-for-AD-Workshop/main/AD_Synthetic_Data.xlsx'  # URL to the data file in the repo
df = pd.read_excel(data_url, sheet_name='Sheet1')  # Reads the Excel file from URL into a DataFrame
print("Data preview (first 5 rows):")  # Prints a message
display(df.head())  # Shows the first 5 rows in a nice table format

## Step 3: Define Inputs and Output

**Description for Beginners**: Inputs (features) are the variables that influence biogas production, like temperature or substrate levels. The output (label) is what we want to predict: 'q_gas' (biogas flow rate). We separate them into X (inputs) and y (output).

**Features Explained** (based on typical AD models like ADM1):
- `S_*`: Soluble substrates (e.g., `S_su` = sugars, `S_aa` = amino acids)—these are broken down by microbes.
- `X_*`: Particulate biomass (e.g., `X_su` = sugar degraders).
- `Q`: Inflow rate, `T (C)`: Temperature in Celsius—key for microbial activity.
- Others: Ions, inerts, etc.

**Why Do This?** The ANN learns to map inputs to output, like 'high temperature + good substrates = more biogas'.

**Tips**: Update the 'features' list to match the full-form column names (e.g., 'Carbohydrates' instead of 'S_su') after modifying your data file.

In [ ]:
features = ['S_su', 'S_aa', 'S_fa', 'S_va', 'S_bu', 'S_pro', 'S_ac', 'S_IC', 'S_IN', 
            'S_I', 'Q', 'T (C)', 'X_xc', 'X_ch', 'X_pr', 'X_li', 'X_su', 'X_aa', 
            'X_fa', 'X_c4', 'X_pro', 'X_ac', 'X_h2', 'X_I', 'S_cation', 'S_anion']  # List of input column names

label = 'q_gas'  # The output column name

X = df[features]  # Selects input columns from the DataFrame
y = df[label]  # Selects the output column

## Step 4: Feature Selection Using Mutual Information

**Background on Feature Selection**: Feature selection is the process of identifying and selecting a subset of relevant features (inputs) for use in model construction. It helps improve model performance by reducing overfitting, shortening training times, and enhancing generalisation by removing irrelevant or redundant data. In data-driven modelling like this, with potentially many AD parameters, feature selection ensures the model focuses on the most impactful variables.

![Feature Selection Schematic](https://raw.githubusercontent.com/rm01243/Data-Driven-Modelling-for-AD-Workshop/main/Feature%20Selection%20Image.png)

**Description for Beginners**: Not all inputs are equally important. Mutual Information (MI) scores how much each feature 'informs' the output. A high score means the feature strongly relates to biogas production. We rank them and select the top 10 to simplify the model and focus on key variables.

**Why Do This?** Too many features can confuse the model (overfitting) or slow it down. This step identifies 'VIPs' like temperature, which often impacts AD heavily.

**Tips**: Change `top_n` to select more/fewer features. Run this to see which AD parameters matter most in your data.

In [ ]:
mi_scores = mutual_info_regression(X, y, random_state=42)  # Calculates MI scores between features and output
mi_df = pd.DataFrame({'Feature': features, 'Mutual_Information': mi_scores})  # Creates a table with features and scores
mi_df.sort_values(by='Mutual_Information', ascending=False, inplace=True)  # Sorts the table by score, highest first
print("Feature importance based on Mutual Information:")  # Prints a message
display(mi_df)  # Shows the sorted table

# Select top N features
top_n = 10  # Number of top features to keep; you can change this (e.g., to 5 for simpler model)
top_features = mi_df['Feature'].head(top_n).tolist()  # Gets the names of top features
print(f"Top {top_n} features selected for training: {top_features}")  # Prints the selected features

X_top = df[top_features]  # Updates X to only include top features

# Feature Importance Plot (bar chart for visualisation)
top_mi_features = mi_df.head(top_n).sort_values(by='Mutual_Information')  # Selects and sorts for horizontal bar
plt.figure(figsize=(10,6))  # Sets plot size
plt.barh(top_mi_features['Feature'], top_mi_features['Mutual_Information'], color='skyblue')  # Creates horizontal bar chart
plt.xlabel('Mutual Information Score')  # Label for x-axis
plt.title(f'Top {top_n} Features Based on Mutual Information')  # Plot title
plt.tight_layout()  # Adjusts layout to fit labels
plt.show()  # Displays the plot

## Step 5: Data Preprocessing

**Description for Beginners**: Raw data varies in scale (e.g., temperature in 30-60°C, concentrations in 0-100). We 'scale' everything to 0-1 range so no feature dominates. Then, split into training (80%) for learning and testing (20%) for validation. No shuffling because AD data is time-series (order matters).

**Why Do This?** ANNs learn better from normalised data. Splitting prevents 'cheating'—the model shouldn't see test data during training.

**Tips**: Scaling is like converting all units to 'apples' for fair comparison. If your data has outliers, consider other scalers.

In [ ]:
scaler_X = MinMaxScaler()  # Creates a scaler for inputs
X_scaled = scaler_X.fit_transform(X_top)  # Fits and scales inputs to 0-1

scaler_y = MinMaxScaler()  # Creates a scaler for output
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # Scales output (reshapes to 2D for scaler)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False  # 20% test, no shuffle for time order, random_state for reproducibility
)

## Step 6: Build the ANN Model

**Description for Beginners**: An ANN is like a layered cake: input layer takes data, hidden layers process it (learning patterns), output layer gives prediction. We use 'relu' activation to handle non-linear relationships (AD processes aren't straight lines). Compile sets how the model learns (optimizer adjusts weights, loss measures error).

![Simple Feedforward Neural Network](https://learnopencv.com/wp-content/uploads/2017/10/mlp-diagram.jpg)

#### How Does an ANN Work?
Initially, the weights of the network are randomly assigned. When input is provided to the input layer, it progresses forward, with hidden layers receiving the input combined with weights. This continues until the output layer produces a result. The result is compared to the actual value, and backpropagation adjusts the weights to improve accuracy. Neurons in the layers handle individual learning through activation functions, which determine if a signal passes based on the input from the previous layer.

#### Activation Functions
Activation functions are crucial for ANNs to learn complex, non-linear mappings between inputs and outputs. They introduce non-linearity to the network. Common types include:

- **Sigmoid (Logistic)**: Outputs between 0 and 1, S-shaped curve. Easy to use but suffers from vanishing gradients, slow convergence, and non-zero-centred output.
- **Tanh (Hyperbolic Tangent)**: Outputs between -1 and 1, zero-centred for easier optimisation, but still prone to vanishing gradients.
- **ReLU (Rectified Linear Unit)**: Simple and efficient (max(0, x)), avoids vanishing gradients, and speeds up convergence. Widely used in hidden layers but can cause 'dead neurons'; Leaky ReLU addresses this with a small slope for negative values.

For output layers, use Softmax for classification (probabilities) or linear for regression.

#### What Happens Without an Activation Function?
Without activation functions, the output would be a simple linear function, limiting the network to modelling basic relationships like linear regression. This lacks the power to handle complex, non-linear data such as images or time-series, making ANNs ineffective for advanced tasks. Activation functions enable the network to learn intricate patterns in high-dimensional data.

**Why Do This?** This structure is simple yet powerful for prediction tasks like biogas output.

**Tips**: Neurons (64, 32) are like brain cells; more can learn complex patterns but risk overfitting. Experiment by adding layers or changing the number of neurons!

In [ ]:
from tensorflow.keras.layers import Dense, Input

model = Sequential([
    Input(shape=(X_train.shape[1],)),              # Proper input layer
    Dense(64, activation='relu'),                  # First hidden layer
    Dense(32, activation='relu'),                  # Second hidden layer
    Dense(1)                                       # Output layer (regression)
])

model.compile(optimizer='adam', loss='mean_squared_error')

## Step 7: Train the Model

**Description for Beginners**: Training is where the ANN adjusts its internal weights to minimise errors. It 'sees' the training data multiple times (epochs). Validation uses test data to check progress. Loss should decrease over epochs.

**Parameters Explained**:
- `epochs=50`: How many times the model reviews the data.
- `batch_size=32`: Processes data in groups of 32 for efficiency.
- `verbose=1`: Shows progress bar.

**Why Do This?** Like practising a skill—the model gets better with iterations.

**Tips**: If loss stops decreasing, it's converged. Increase epochs if needed, but watch for overfitting (train loss low, val loss high).

In [ ]:
start_time = time.time()  # Starts timer
history = model.fit(X_train, y_train, epochs=50, batch_size=32, 
                    validation_data=(X_test, y_test), verbose=1)  # Trains the model, saves history for plots
end_time = time.time()  # Ends timer
print(f"Training time: {end_time - start_time:.2f} seconds")  # Prints time taken

## Step 8: Make Predictions and Evaluate

**Description for Beginners**: Use the trained model to predict on train/test data. Inverse scale back to original units. Metrics: R² (0-1, higher better—fit quality), RMSE (error magnitude, lower better).

**Background on Overfitting and Underfitting**: Overfitting occurs when the model learns the training data too well, including noise, and performs poorly on new data. Underfitting happens when the model is too simple and doesn't capture patterns in the data. You can detect overfitting if training metrics are good but test metrics are poor; underfitting if both are poor.

**Why Do This?** To quantify how accurate the model is. Good test metrics mean it generalises to new data.

**Tips**: R² > 0.8 is decent for AD modelling. If low, try more data or tweak model.

In [ ]:
y_train_pred = model.predict(X_train)  # Predicts on training data
y_test_pred = model.predict(X_test)  # Predicts on test data

# Inverse scaling to original units
y_train_orig = scaler_y.inverse_transform(y_train)  # Unscales training output
y_train_pred_orig = scaler_y.inverse_transform(y_train_pred)  # Unscales training predictions
y_test_orig = scaler_y.inverse_transform(y_test)  # Unscales test output
y_test_pred_orig = scaler_y.inverse_transform(y_test_pred)  # Unscales test predictions

# Calculate metrics
r2_train = r2_score(y_train_orig, y_train_pred_orig)  # R² for train
rmse_train = np.sqrt(mean_squared_error(y_train_orig, y_train_pred_orig))  # RMSE for train
r2_test = r2_score(y_test_orig, y_test_pred_orig)  # R² for test
rmse_test = np.sqrt(mean_squared_error(y_test_orig, y_test_pred_orig))  # RMSE for test

print(f'Training R²: {r2_train:.4f} (how well it fits train data, 1=perfect), RMSE: {rmse_train:.4f} (average error)')  # Prints train metrics
print(f'Test R²: {r2_test:.4f}, RMSE: {rmse_test:.4f}')  # Prints test metrics

## Step 9: Visualise Results

**Description for Beginners**: Plots show actual vs. predicted biogas (lines should match closely) and loss over epochs (should decrease). This visually confirms performance.

**Why Do This?** Numbers are good, but graphs reveal trends, like if predictions follow AD fluctuations.

**Tips**: If lines don't overlap on test set, the model may overfit—try fewer features or more data.

In [ ]:
# Create a single figure with three subplots side by side
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns, wide figure to fit legends

# Plot 1: Training Set Actual vs Predicted
axs[0].plot(y_train_orig, label='Actual Train', color='blue')
axs[0].plot(y_train_pred_orig, label='Predicted Train', color='orange')
axs[0].set_title('Training Set: Actual vs Predicted')
axs[0].set_ylabel('q_gas')
axs[0].set_xlabel('Time Steps')
axs[0].legend(loc='upper left')

# Plot 2: Test Set Actual vs Predicted
axs[1].plot(y_test_orig, label='Actual Test', color='blue')
axs[1].plot(y_test_pred_orig, label='Predicted Test', color='orange')
axs[1].set_title('Test Set: Actual vs Predicted')
axs[1].set_ylabel('q_gas')
axs[1].set_xlabel('Time Steps')
axs[1].legend(loc='upper left')

# Plot 3: Model Loss During Training
axs[2].plot(history.history['loss'], label='Train Loss', color='green')
axs[2].plot(history.history['val_loss'], label='Validation Loss', color='red')
axs[2].set_title('Model Loss During Training')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('Loss')
axs[2].legend(loc='upper left')

plt.tight_layout()  # Adjusts layout to prevent overlap
plt.show()  # Displays all three plots in one row

## Wrapping Up
You've completed the workshop! You've loaded data, selected key features, preprocessed it, built and trained an ANN, and evaluated its performance with metrics and plots.

**Next Steps**:
- Apply this to your real AD plant data (clean it first if needed).
- Experiment: Change top_n, epochs, number of neurons, or add layers to improve results.
- Explore methods to automate tuning of model parameters (hyperparameters).